<a href="https://colab.research.google.com/github/SandroP10/Sales-Forecast-Project/blob/main/ARIMA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ca_training_data_hs23/final_feat.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15567 entries, 0 to 15566
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   article_name        15567 non-null  float64
 1   date_id             15567 non-null  int64  
 2   year                15567 non-null  int64  
 3   month               15567 non-null  int64  
 4   week                15567 non-null  int64  
 5   boss_thema_id       15567 non-null  int64  
 6   promo_01            15567 non-null  int64  
 7   promo_02            15567 non-null  int64  
 8   promo_03            15567 non-null  int64  
 9   promo_04            15567 non-null  int64  
 10  promo_05            15567 non-null  int64  
 11  sales               15567 non-null  float64
 12  zscore              15567 non-null  float64
 13  sales_lag1          15567 non-null  float64
 14  sales_lag2          15567 non-null  float64
 15  month_sin           15567 non-null  float64
 16  mont

In [ ]:
df.head()

,article_name,date_id,year,month,week,boss_thema_id,promo_01,promo_02,promo_03,promo_04,...,sales,zscore,sales_lag1,sales_lag2,month_sin,month_cos,lag1_sales_7d_mean,lag2_sales_30d_std,days_since_promo,month_promo01
0,13.0,201926,2019,6,26,5,0,0,0,0,...,1209.0,-1.147814,0.0,0.0,1.224647e-16,-1.000000,0.0,720.17804,0,0
1,13.0,201927,2019,7,27,5,0,0,0,0,...,11655.6,1.984810,1209.0,0.0,-5.000000e-01,-0.866025,0.0,720.17804,0,0
2,13.0,201928,2019,7,28,5,0,0,0,0,...,11312.0,1.771019,11655.6,1209.0,-5.000000e-01,-0.866025,0.0,720.17804,0,0
3,13.0,201929,2019,7,29,5,0,0,0,0,...,10621.0,1.571384,11312.0,11655.6,-5.000000e-01,-0.866025,0.0,720.17804,0,0
4,13.0,201930,2019,7,30,5,0,0,0,0,...,11655.6,2.355768,10621.0,11312.0,-5.000000e-01,-0.866025,0.0,720.17804,0,0


In [ ]:
# Model Development
# Time series models

y = df['sales']

# Fit ARIMA model
model = ARIMA(endog=y, order=(2,1,2))
model_fit = model.fit()

# Make predictions
predictions = model_fit.predict(start=len(df), end=len(df)+4)

In [ ]:
# Calculate error metrics
arima_mse = mean_squared_error(y[-5:], predictions)
arima_mape = mean_absolute_percentage_error(y[-5:], predictions)

In [ ]:
print(predictions, arima_mse, arima_mape)

15567    11173.334123
15568    11103.029113
15569    11111.190015
15570    11104.455385
15571    11107.723574
Name: predicted_mean, dtype: float64 1114663.7921189817 0.08170927749774014


In [ ]:
# Convert Series to numpy array
predictions = predictions.values


print(predictions.shape)


(5,)


In [ ]:
pred_df = pd.DataFrame(predictions, columns=['predictions(ARIMA)'])

output_path = '/content/drive/MyDrive/ca_training_data_hs23'

pred_df.to_csv(os.path.join(output_path,'model_predictions.csv'), index=False)


- Under 10% MAPE is generally acceptable.

- The predictions seem reasonable and in line with previous values
- The MSE is high, so the model is not very precise
- The MAPE is within a decent range for a time series model

###Next steps:
- tuning the ARIMA parameters (p,d,q) using a grid search to find the optimal combination
- Add more relevant predictors like promotions, prices, etc if available
- Try using regularization or ensembling to reduce overfitting and improve precision
- Check performance on test set too.

In [ ]:
import pickle

# Save model
with open('ARIMAmodel.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
# Load model
with open('ARIMAmodel.pkl', 'rb') as file:
   loaded_model = pickle.load(file)

In [ ]:
#!pip install pmdarima

In [ ]:
""""
from pmdarima.model_selection import train_test_split
from pmdarima import auto_arima

# Wrapper class
from sklearn.base import BaseEstimator, RegressorMixin

class ARIMAWrapper(BaseEstimator, RegressorMixin):

    def __init__(self, order=None):
        self.order = order
        self.model = None

    def fit(self, X, y):
        self.model = auto_arima(y, order=self.order)
        self.model.fit(X)
        return self

    def predict(self, X):
        return self.model.predict(n_periods=X.shape[0])

    def get_params(self, deep=True):
        return {"order": self.order}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

# Initialize wrapper
arima = ARIMAWrapper()

# Use with grid search
gs = GridSearchCV(arima, params, cv=3)


from sklearn.model_selection import train_test_split

X = df.drop(['sales'], axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

gs.fit(X_train, y_train)

best_model = gs.best_estimator_
"""

'"\nfrom pmdarima.model_selection import train_test_split\nfrom pmdarima import auto_arima\n\n# Wrapper class\nfrom sklearn.base import BaseEstimator, RegressorMixin\n\nclass ARIMAWrapper(BaseEstimator, RegressorMixin):\n\n    def __init__(self, order=None):\n        self.order = order\n        self.model = None\n\n    def fit(self, X, y):\n        self.model = auto_arima(y, order=self.order)\n        self.model.fit(X)\n        return self\n\n    def predict(self, X):\n        return self.model.predict(n_periods=X.shape[0])\n\n    def get_params(self, deep=True):\n        return {"order": self.order}\n\n    def set_params(self, **parameters):\n        for parameter, value in parameters.items():\n            setattr(self, parameter, value)\n        return self\n\n# Initialize wrapper\narima = ARIMAWrapper()\n\n# Use with grid search\ngs = GridSearchCV(arima, params, cv=3)\n\n\nfrom sklearn.model_selection import train_test_split\n\nX = df.drop([\'sales\'], axis=1)\ny = df[\'sales\']\n\n

In [ ]:
"""

import itertools

# Parameters to test
p = [0, 1, 2]
d = [0, 1]
q = [0, 1, 2]

# Generate combinations
parameters = itertools.product(p, d, q)

# Iterate through combinations
best_mse = float("inf")
best_params = None
for param in parameters:
    order = param
    model = ARIMA(order=order)
    model.fit(X_train, y_train)

    mse = mean_squared_error(y_test, model.predict(X_test))
    if mse < best_mse:
        best_mse = mse
        best_params = order

print("Best Parameters:", best_params)
print("Lowest MSE:", best_mse)
"""

'\n\nimport itertools\n\n# Parameters to test\np = [0, 1, 2]\nd = [0, 1]\nq = [0, 1, 2]\n\n# Generate combinations\nparameters = itertools.product(p, d, q)\n\n# Iterate through combinations\nbest_mse = float("inf")\nbest_params = None\nfor param in parameters:\n    order = param\n    model = ARIMA(order=order)\n    model.fit(X_train, y_train)\n\n    mse = mean_squared_error(y_test, model.predict(X_test))\n    if mse < best_mse:\n        best_mse = mse\n        best_params = order\n\nprint("Best Parameters:", best_params)\nprint("Lowest MSE:", best_mse)\n'

In [ ]:
"""

# Model Optimization

# ARIMA GridSearch
p_values = [0, 1, 2]
d_values = [0, 1]
q_values = [0, 1, 2]

params = {'order': [(p,d,q) for p in p_values for d in d_values for q in q_values]}

gs = GridSearchCV(ARIMA(endog=y), params, scoring='neg_mean_squared_error', cv=3)
gs.fit(y)

arima_tuned = gs.best_estimator_

# Final Forecasts

# Retrain final ARIMA model
arima_final = ARIMA(order=arima_tuned.order)
arima_final.fit(y)

arima_forecasts = arima_final.predict(len(y), len(y)+4)


print(arima_forecasts)

"""

"\n\n# Model Optimization\n\n# ARIMA GridSearch\np_values = [0, 1, 2]\nd_values = [0, 1]\nq_values = [0, 1, 2]\n\nparams = {'order': [(p,d,q) for p in p_values for d in d_values for q in q_values]}\n\ngs = GridSearchCV(ARIMA(endog=y), params, scoring='neg_mean_squared_error', cv=3)\ngs.fit(y)\n\narima_tuned = gs.best_estimator_\n\n# Final Forecasts\n\n# Retrain final ARIMA model\narima_final = ARIMA(order=arima_tuned.order)\narima_final.fit(y)\n\narima_forecasts = arima_final.predict(len(y), len(y)+4)\n\n\nprint(arima_forecasts)\n\n"

In [ ]:
"""

N = pd.read_csv('/content/drive/MyDrive/ca_training_data_hs23/final_feat.csv')

N.info()
"""

"\n\nN = pd.read_csv('/content/drive/MyDrive/ca_training_data_hs23/final_feat.csv')\n\nN.info()\n"